In [2]:
from pyarrow import flight, Table
from pyarrow.flight import FlightClient
import requests


## Function to Retrieve PAT Token from Dremio
def get_token(uri, payload):
    # Make the POST request
    response = requests.post(uri, json=payload)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        # Extract the token
        return data.get("token", "")
    else:
        print("Failed to get a valid response. Status code:", response.status_code)


## Arrow Endpoint
location = "grpc://localhost:32010"

## Username and Password for Dremio Account
username = "deverty"
password = "09102003sK"

## Dremio REST API URL To Login and Get Token
uri = "http://localhost:9047/apiv2/login"

## Payload for Get Token Requests
payload = {"userName": username, "password": password}

## Auth Token
token = get_token(uri, payload)
print(token)

## Headers for Arrow Requests
headers = [(b"authorization", f"bearer {token}".encode("utf-8"))]

## Query
query = """
SELECT * FROM nessie.students
"""

## Create Arrow Flight Client
client = FlightClient(location=(location), disable_server_verification=False)

## Create Flight Call Options
options = flight.FlightCallOptions(headers=headers)

## Send Query
flight_info = client.get_flight_info(
    flight.FlightDescriptor.for_command(query), options
)

## Get Query Results
results:flight.FlightStreamReader = client.do_get(flight_info.endpoints[0].ticket, options)
table: Table = results.read_all()
print(table)

nmvdbsfbvaao8depntohphb3he
pyarrow.Table
Личный номер: string
ФИО: string
Группа: string
Финансирование: string
Пол: string
Город постоянной регистрации: string
----
Личный номер: [["22И7571","23И6772","20И6411","22И8185","23И9648",...,"21И8654","21И3549","21И9010","21И6328","22И4538"],["20И1158","23И8458","22И0667","21И5237","20И0860",...,"21И8504","22И7019","23И7399","22И2339","23И7225"],...,["23И0868","23И6382","21И2575","20И5043","21И7657",...,"22И0764","22И6360","22И4201","20И8511","23И3728"],["22И8998","21И0587","23И3699","21И9628","21И8497",...,"23И4512","20И1281","21И6217","23И7133","20И3496"]]
ФИО: [["Афанасий Захарьевич Уваров","Нина Федоровна Лукина","Петров Модест Давидович","Баранова Анжела Николаевна","Василиса Болеславовна Белова",...,"Раиса Анатольевна Денисова","Зиновий Гордеевич Воронцов","Василиса Артемовна Мартынова","Селиверст Афанасьевич Колобов","Силантий Глебович Осипов"],["Тамара Захаровна Комарова","Дарья Антоновна Чернова","Игнатова Ия Мироновна","Федотов Афи

In [ ]:
from pyarrow import flight
from pyarrow.flight import FlightClient
import pyarrow.dataset as ds
import pandas as pd

class DremioConnection:
    def __init__(self, token, location):
        self.token = token
        self.location = location
        self.headers = [
    (b"authorization", f"bearer {token}".encode("utf-8"))
    ]
        self.client = FlightClient(location=(location))
       
    def query(self, query, client, headers):
        ## Options for Query
        options = flight.FlightCallOptions(headers=headers)
       
        ## Get ticket to for query execution, used to get results
        flight_info = client.get_flight_info(flight.FlightDescriptor.for_command(query), options)
   
        ## Get Results (Return Value a FlightStreamReader)
        results = client.do_get(flight_info.endpoints[0].ticket, options)
        return results
       
    # Returns a FlightStreamReader
    def toArrow(self, query):
        return self.query(query, self.client, self.headers)